# Creating supercells with pymatgen

The [Pymatgen](https://pymatgen.org/) python library allows to setup
solid-state calculations using a flexible set of classes as well as an API
to an online data base of structures. Its `Structure` and `Lattice`
objects are directly supported by the DFTK `load_atoms` and `load_lattice`
functions, such that DFTK may be readily used to run calculation on systems
defined in pymatgen. Using the `pymatgen_structure` function a conversion
from DFTK to pymatgen structures is also possible. In the following we
use this to create a silicon supercell and find its LDA ground state
using direct minimisation. To run this example Julia's `PyCall` package
needs to be able to find an installation of `pymatgen`.

First we setup the silicon lattice in DFTK.

In [1]:
using DFTK

a = 10.263141334305942  # Lattice constant in Bohr
lattice = a / 2 .* [[0 1 1.]; [1 0 1.]; [1 1 0.]]
Si = ElementPsp(:Si, psp=load_psp("hgh/lda/Si-q4"))
atoms = [Si => [ones(3)/8, -ones(3)/8]];

Next we make a `[2, 2, 2]` supercell using pymatgen

In [2]:
pystruct = pymatgen_structure(lattice, atoms)
pystruct.make_supercell([2, 2, 2])
lattice = load_lattice(pystruct)
atoms = [Si => [s.frac_coords for s in pystruct.sites]];

┌ Warning: `vendor()` is deprecated, use `BLAS.get_config()` and inspect the output instead
│   caller = npyinitialize() at numpy.jl:67
└ @ PyCall /home/runner/.julia/packages/PyCall/L0fLP/src/numpy.jl:67


Setup an LDA model and discretize using
a single k-point and a small `Ecut` of 5 Hartree.

In [3]:
model = model_LDA(lattice, atoms)
basis = PlaneWaveBasis(model; Ecut=5, kgrid=(1, 1, 1))

PlaneWaveBasis discretization:
    Ecut                 : 5.0 Ha
    fft_size             : (32, 32, 32), 32768 total points
    kgrid type           : Monkhorst-Pack
    kgrid                : [1, 1, 1]
    num. irred. kpoints  : 1

    Discretized Model(lda_xc_teter93, 3D):
        lattice (in Bohr)    : [0         , 10.2631   , 10.2631   ]
                               [10.2631   , 0         , 10.2631   ]
                               [10.2631   , 10.2631   , 0         ]
        unit cell volume     : 2162.1 Bohr³
    
        atoms                : Si₁₆
        atom potentials      : ElementPsp(Si, psp="hgh/lda/si-q4")
    
        num. electrons       : 64
        spin polarization    : none
        temperature          : 0 Ha
    
        terms                : Kinetic()
                               AtomicLocal()
                               AtomicNonlocal()
                               Ewald()
                               PspCorrection()
                               

Find the ground state using direct minimisation (always using SCF is boring ...)

In [4]:
scfres = direct_minimization(basis, tol=1e-5);

Iter     Function value   Gradient norm 
     0     1.125074e+02     1.420811e+00
 * time: 0.5044059753417969
     1     1.050562e+01     7.624286e-01
 * time: 1.1787800788879395
     2    -1.261328e+01     9.294406e-01
 * time: 1.2914700508117676
     3    -3.428434e+01     6.716754e-01
 * time: 1.3742001056671143
     4    -4.791102e+01     5.073717e-01
 * time: 1.4725050926208496
     5    -5.699056e+01     1.832077e-01
 * time: 1.5712120532989502
     6    -5.980367e+01     1.298000e-01
 * time: 1.6580941677093506
     7    -6.090675e+01     5.582211e-02
 * time: 1.7181930541992188
     8    -6.133598e+01     4.128990e-02
 * time: 1.7783010005950928
     9    -6.157352e+01     4.463372e-02
 * time: 1.8397040367126465
    10    -6.177122e+01     3.536237e-02
 * time: 1.9001071453094482
    11    -6.193035e+01     2.441184e-02
 * time: 1.975492000579834
    12    -6.201608e+01     1.941735e-02
 * time: 2.036701202392578
    13    -6.208886e+01     1.568227e-02
 * time: 2.098148107528

In [5]:
scfres.energies

Energy breakdown (in Ha):
    Kinetic             25.7671070
    AtomicLocal         -18.8557712
    AtomicNonlocal      14.8522674
    Ewald               -67.1831486
    PspCorrection       -2.3569765
    Hartree             4.8485379 
    Xc                  -19.3336823

    total               -62.261666459266